In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Dec 12 22:27:52 2014

@author: LukasHalim
Forked by @edridgedsouza

2) For each post, determine earliest failure
- Sort the comments for each post in ascending order
- Create a variable for the comment number within each post
- In cases where there is a failure, identify the first failing comment
- In cases where there is not a failure, identify the final comment for the post


"""

import pandas as pd
import matplotlib.pyplot as plt
#import csv

import sqlite3
from godwin import Scraper, Database

db = Database('Godwin.db')
# db.reset_db()
s = Scraper(db)

In [ ]:
s.scrape_top_subreddits(time_filter='month', limit=100)

Scraping from /r/politicalcompassmemes: 0it [00:00, ?it/s]
Scraped 1 of 200 subreddits


Subreddit politicalcompassmemes forbidden


Scraping from /r/europe: 100it [00:02, 46.23it/s]
Scraped 2 of 200 subreddits
Scraping from /r/teenagers: 100it [00:02, 43.56it/s]
Scraped 3 of 200 subreddits
Scraping from /r/witcher: 100it [00:02, 42.65it/s]
Scraped 4 of 200 subreddits
Scraping from /r/mechanicalkeyboards: 100it [00:02, 44.00it/s]
Scraped 5 of 200 subreddits
Scraping from /r/instagram: 100it [00:01, 90.67it/s]
Scraped 6 of 200 subreddits
Scraping from /r/moviedetails: 100it [00:01, 62.30it/s]
Scraped 7 of 200 subreddits
Scraping from /r/fo76: 100it [00:01, 50.45it/s]
Scraped 8 of 200 subreddits
Scraping from /r/whatcouldgowrong: 100it [00:02, 40.32it/s]
Scraped 9 of 200 subreddits
Scraping from /r/totalwar: 100it [00:01, 59.62it/s]
Scraped 10 of 200 subreddits
Scraping from /r/fortnitebr: 100it [00:02, 37.80it/s]
Scraped 11 of 200 subreddits
Scraping from /r/cringe: 100it [00:01, 70.91it/s]
Scraped 12 of 200 subreddits
Scraping from /r/barstoolsports: 100it [00:01, 74.13it/s]
Scraped 13 of 200 subreddits
Scraping fro

Subreddit cringetopia forbidden


Scraping from /r/facepalm: 100it [00:02, 41.87it/s]
Scraped 15 of 200 subreddits
Scraping from /r/books: 100it [00:02, 41.50it/s]
Scraped 16 of 200 subreddits
Scraping from /r/pokemon: 100it [00:02, 42.74it/s]
Scraped 17 of 200 subreddits
Scraping from /r/apexlegends: 100it [00:02, 40.55it/s]
Scraped 18 of 200 subreddits
Scraping from /r/dndnext: 100it [00:01, 65.54it/s]
Scraped 19 of 200 subreddits
Scraping from /r/trashy: 0it [00:00, ?it/s]
Scraped 20 of 200 subreddits


...

Scraping from /r/explainlikeimfive: 100it [00:00, 103.00it/s]
Scraped 99 of 200 subreddits
Scraping from /r/watchpeopledieinside: 100it [00:01, 62.42it/s]
Scraped 100 of 200 subreddits
Scraping from /r/art: 100it [00:02, 38.73it/s]
Scraped 101 of 200 subreddits
Scraping from /r/gifs: 100it [00:02, 49.88it/s]
Scraped 102 of 200 subreddits
Scraping from /r/memes: 100it [00:03, 31.74it/s]
Scraped 103 of 200 subreddits
Scraping from /r/minecraft: 100it [00:02, 39.85it/s]
Scraped 104 of 200 subreddits
Scraping from /r/gtaonline: 100it [00:02, 36.52it/s]
Scraped 105 of 200 subreddits
Scraping from /r/todayilearned: 100it [00:01, 50.55it/s]
Scraped 106 of 200 subreddits
Scraping from /r/aww: 100it [00:03, 30.31it/s]
Scraped 107 of 200 subreddits
Scraping from /r/gaming: 100it [00:02, 37.74it/s]
Scraped 108 of 200 subreddits
Scraping from /r/animalcrossing: 100it [00:02, 35.59it/s]
Scraped 109 of 200 subreddits
Scraping from /r/videos: 100it [00:02, 36.97it/s]
Scraped 110 of 200 subreddits
Scr

Subreddit iphone forbidden


Scraping from /r/damnthatsinteresting: 100it [02:48,  1.69s/it]
Scraped 144 of 200 subreddits
Scraping from /r/pokemongo: 0it [00:00, ?it/s]
Scraped 145 of 200 subreddits


Subreddit pokemongo forbidden


Scraping from /r/parenting: 22it [00:32,  1.34s/it]

In [ ]:

comments_df = pd.read_sql("select * from comment",conn)
g = comments_df.groupby('post_id')
#Sort the comments for each post in ascending order
comments_df['RN'] = g['comment_created'].rank(method='first')
#Create a variable for the comment number within each post
comments_with_nazi_df = comments_df[comments_df.nazi_in_comment == 1]
#Identify posts where there is a mention of Nazi
nazi_posts = comments_with_nazi_df['post_id'].unique()

#In cases where there is a comparison with Nazis or Hitler, identify the first comment 
#where the comparison is made
mins = comments_with_nazi_df.groupby('post_id')['RN'].idxmin()
first_nazi_comment = comments_with_nazi_df.loc[mins]

right_censored_posts = comments_df[comments_df.post_id.isin(nazi_posts) == False]
maxes = right_censored_posts.groupby('post_id')['RN'].idxmax()
final_comment = right_censored_posts.loc[maxes]

#combine the censored posts with those where a comparison is made
concatenated = pd.concat([first_nazi_comment,final_comment])

T = concatenated['RN']
E = concatenated['nazi_in_comment']

from lifelines import KaplanMeierFitter
kmf = KaplanMeierFitter()
kmf.fit(T, event_observed=E)
kmf.plot()

plt.xlim(0,2000);
plt.title("Reddit Post Lifespan Prior to Mention of Nazi or Hitler");
plt.xlabel("Comments")
plt.ylabel("Fraciton of Posts Without Mention of Hitler or Nazis")